In [1]:
import numpy as np
import time

In [2]:
def distance(i,j):
    """Returns the distance between two points of R²"""

    d_2 = (i[0]-j[0])**2 + (i[1]-j[1])**2
    return np.sqrt(d_2)

In [3]:
def check_neighbors(P, i, j):
    """Returns True if the points i,j of R² are neighbors according to the list of points P.
    Assumes that i!=j. Complexity is O(len(P))"""
    
    d_ij = distance(i,j)
    
    for k in P:
        if (i==k or k==j) : continue
        if(max(distance(i,k), distance(k,j)) < d_ij): return False
    else:   # If the for loop isn't broken
        return True

In [4]:
def find_closest(P,i):
    """Finds the closest point to i in the list P in O(len(P))"""
    
    d_min = float('inf')
    ind_closest = 0
    
    for l, j in enumerate(P):
        if i==j: continue
        d = distance(i, j)
        if d < d_min:
            closest = j
            d_min = d
            ind_closest = l
            
    return(closest,ind_closest)

In [5]:
def naive_neighbors(P):
    """Takes as entry a list P of points in R² and compute naively the graph of the neighbors, e.g 
    finds all neighbors for each point in P in a simple but non-efficient way. Complexity is cubic in len(P)"""

    graph = []

    for m,i in enumerate(P):
        graph.append(set())    # Set of neighbors of point i
        for j in P: 
            if (i==j) : continue
            if check_neighbors(P, i, j): graph[m].add(j)
            
    return graph

In [6]:
def find_neighbors_linear(P, p):
    """Uses the method described in Task 6 to achieve a quadratic complexity to find the graph,
    but only applied to a single vertex."""
    
    P_copy = P.copy()
    P_copy.remove(p)
    
    potential_neighbors = []
    true_nghbr = set()
    
    # Finding potential neighbors    
    while (len(P_copy) > 0):
        q, index = find_closest(P_copy, p)              
        potential_neighbors.append(q)
        P_copy.pop(index)
        
        for index, r in enumerate(P_copy):
            if (distance(p,r) > distance(r,q)): P_copy.pop(index)
            
    # We now need to check whether the elements are true neighbors of p
    for j in potential_neighbors:
        if check_neighbors(P, p, j): true_nghbr.add(j)

    # print("potential :", potential_neighbors)
    # print("true neighbors : ", true_nghbr)
    
    return true_nghbr

In [7]:
def quadra_algo_neighbors(P):
    """Enhanced version of the naive algorithm, finding the neighbors graph in quadratic complexity."""
    
    graph = []
    for p in P:
        graph.append(find_neighbors_linear(P, p))  
        
    return graph

In [8]:
def update_graph(graph, P, p):
    """Updates the graph with the addition of point p to the set of vertices P.
    We need to find the neighbors of p and to check if the previous neighbors remain."""
    
    new_P = P + [p]
    
    # We first check the existing edges
    for k in range(len(P)):
      i = P[k]
      for j in graph[k].copy():
          if not(check_neighbors(new_P, i, j)): graph[k].remove(j)
          
    
    # Then we find the neighbors of p
    # Not forgetting to add the new edges to both vertices set of neighbors
    nghbr_p = find_neighbors_linear(new_P, p)
    graph.append(nghbr_p)
    for i in nghbr_p:
        graph[P.index(i)].add(p) # This line is costly beacuse we have to find the position of the neighbor in P
        
    # return graph

In [9]:
def graph_via_update(P):
    """Creates the neighborhood graph using the update_graph function. len(P) should be greater than 2"""
    
    graph = [set()]
    
    for i in range(1,len(P)):
        update_graph(graph, P[:i], P[i])
        
    return graph